In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

# Define a simple neural network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # First fully connected layer
        self.fc1 = nn.Linear(28*28, 500)  # 28*28 for image pixels, 500 for output nodes
        # Second fully connected layer that takes as input the 500 features from the previous layer
        self.fc2 = nn.Linear(500, 10)  # 10 output nodes for 10 classes of the MNIST digits

    def forward(self, x):
        # Flatten the input data (images)
        x = x.view(-1, 28*28)
        # Apply ReLU non-linearity
        x = F.relu(self.fc1(x))
        # Output layer with a log_softmax activation
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

# Load and transform the MNIST dataset
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=64, shuffle=True)

# Initialize the network and the optimizer
model = Net()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

# Define the training function
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

# Run the training loop over one epoch (you can increase the number of epochs)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
for epoch in range(1):
    train(model, device, train_loader, optimizer, epoch)


In [ ]:
model

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

# Define a simple neural network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

# Load and transform the MNIST dataset
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=64, shuffle=True)

# Initialize the network and the optimizer
model = Net()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

# Define the training function
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    running_loss = 0.0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if batch_idx % 10 == 0:  # Print progress every 10 batches
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

    # Print average loss for the epoch
    average_loss = running_loss / len(train_loader)
    print(f'====> Epoch: {epoch} Average Loss: {average_loss:.4f}')

# Run the training loop over the epochs
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
epochs = 5  # Define the number of epochs
for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
